In [39]:
!pip install langchain chromadb sentence-transformers pypdf


In [40]:
from langchain.document_loaders import PyPDFLoader
from google.colab import files

uploaded = files.upload()
pdf_path = list(uploaded.keys())[0]

loader = PyPDFLoader(pdf_path)
pages = loader.load()
print(f"Total pages loaded: {len(pages)}")

Saving BCSC1807 Syllabus.pdf to BCSC1807 Syllabus (2).pdf
Total pages loaded: 1


In [41]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
docs = splitter.split_documents(pages)

print(f"Total chunks created: {len(docs)}")

Total chunks created: 6


In [42]:
from langchain.embeddings import HuggingFaceEmbeddings

embedding_model = HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2')

In [43]:
from langchain.vectorstores import Chroma

vectorstore = Chroma.from_documents(documents=docs, embedding=embedding_model, persist_directory="./chroma_db")
vectorstore.persist()
print("Embedding stored in Chroma.")

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Embedding stored in Chroma.


/tmp/ipython-input-2329283953.py:4: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectorstore.persist()


In [44]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

In [47]:
from langchain.chains import RetrievalQA
from langchain.llms.base import LLM
from typing import Optional, List

class DummyLLM(LLM):
    def _call(self, prompt: str, stop: Optional[List[str]] = None) -> str:
        return "This is a simulated LLM response to the prompt: " + prompt

    @property
    def _llm_type(self) -> str:
        return "dummy-llm"

qa_chain = RetrievalQA.from_chain_type(llm=DummyLLM(), retriever=retriever)

In [46]:
query = "What is the document about?"
response = qa_chain.run(query)
print("Answer:", response)

Answer: This is a simulated LLM response to the prompt: Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

DEPARTMENT OF COMPUTER ENGINEERING & APPLICATIONS, Institute of Engineering & Technology 
Course Curriculum (w.e.f. Session 2023-24) B. Tech. Computer Science & Engineering

BCSC1807: DESIGN & ANALYSIS OF ALGORITHMS LAB Objective: The objective of this course is that students will understand and implement simple data structures, able demonstrate different sorting and searching techniques. and will be familiar with graphs and their applications.   Credits:01     L-T-P-J:0-0-2-0 Module No. Content  Teaching Hours        I& II          Implementation of sorting algorithms: o Insertion Sort o Bubble Sort o Selection Sort  Divide and conquer approach: o Quick Sort o

structures for a given problem • CO3: Implement the algorithms based on Greedy approach and Dynamic Program

/tmp/ipython-input-3251645052.py:2: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = qa_chain.run(query)
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


In [48]:
questions = [
    "What is the main topic of the document?",
    "Which methods or models are used in the study?",
    "What are the key results or findings?",
    "Is there any future work or limitation mentioned?",
    "What is the purpose or objective of this document?"
]

In [49]:
def ask_with_retriever(qa_chain, retriever, query):
    relevant_docs = retriever.get_relevant_documents(query)
    response = qa_chain.run(query)

    print(f"\n Question: {query}")
    print("\n Retrieved Chunks:")
    for i, doc in enumerate(relevant_docs):
        print(f"\nChunk {i+1}:\n{doc.page_content[:300]}...\n")

    print(" RAG Answer:\n", response)
    return response

In [52]:
def ask_without_retriever(llm, query):
    # Simulate raw LLM answer using full doc context (concat first few chunks)
    context = "\n".join([doc.page_content for doc in docs[:5]])
    prompt = f"Context:\n{context}\n\nQuestion: {query}\nAnswer:"
    answer = llm(prompt)

    print(" Raw LLM Answer:\n", answer)
    return answer

In [51]:
llm=DummyLLM()

In [53]:
for q in questions:
    ask_with_retriever(qa_chain, retriever, q)
    ask_without_retriever(llm, q)
    print("-" * 100)

/tmp/ipython-input-2280290601.py:2: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  relevant_docs = retriever.get_relevant_documents(query)
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
/usr/lo


 Question: What is the main topic of the document?

 Retrieved Chunks:

Chunk 1:
DEPARTMENT OF COMPUTER ENGINEERING & APPLICATIONS, Institute of Engineering & Technology 
Course Curriculum (w.e.f. Session 2023-24) B. Tech. Computer Science & Engineering...


Chunk 2:
BCSC1807: DESIGN & ANALYSIS OF ALGORITHMS LAB Objective: The objective of this course is that students will understand and implement simple data structures, able demonstrate different sorting and searching techniques. and will be familiar with graphs and their applications.   Credits:01     L-T-P-J:...


Chunk 3:
structures for a given problem • CO3: Implement the algorithms based on Greedy approach and Dynamic Programming.    Mapping of Course Outcomes (COs) with Program Outcomes (POs) and Program Specific Outcomes (PSOs):  COs POs/PSOs CO1 PO1,PO2,PO4/PSO1,PSO2,PSO4 CO2 PO1,PO3,PO4/PSO1,PSO2,PSO3 CO3 PO2,P...

 RAG Answer:
 This is a simulated LLM response to the prompt: Use the following pieces of context to answer the

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


In [55]:
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
prompt_template = """
You are a helpful assistant. Use the following document excerpts to answer the question.
Always format the answer in bullet points and cite sources using [source X] format where X is the index of the document.

Context:
{context}

Question:
{question}

Answer in bullet points:
-

[Disclaimer: This is an AI-generated response based on provided context and may not be entirely accurate.]
"""

custom_prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=prompt_template,
)

In [56]:
from langchain.chains.question_answering import load_qa_chain

qa_chain_custom = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type="stuff",
    chain_type_kwargs={"prompt": custom_prompt}
)

In [57]:
response = qa_chain_custom.run("What are the main findings of the document?")
print(response)

This is a simulated LLM response to the prompt: 
You are a helpful assistant. Use the following document excerpts to answer the question.
Always format the answer in bullet points and cite sources using [source X] format where X is the index of the document.

Context:
BCSC1807: DESIGN & ANALYSIS OF ALGORITHMS LAB Objective: The objective of this course is that students will understand and implement simple data structures, able demonstrate different sorting and searching techniques. and will be familiar with graphs and their applications.   Credits:01     L-T-P-J:0-0-2-0 Module No. Content  Teaching Hours        I& II          Implementation of sorting algorithms: o Insertion Sort o Bubble Sort o Selection Sort  Divide and conquer approach: o Quick Sort o

DEPARTMENT OF COMPUTER ENGINEERING & APPLICATIONS, Institute of Engineering & Technology 
Course Curriculum (w.e.f. Session 2023-24) B. Tech. Computer Science & Engineering

structures for a given problem • CO3: Implement the algori

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
